# GyrazeInterface HDF5 File Structure

#### Overview
The `GyrazeInterface` class converts Gkeyll simulation data into the format required by the GYRAZE code, storing multiple datasets in a single HDF5 file.

```text
gkeyll_gyraze_inputs.h5
├── attributes:
│   ├── description: "Gyraze input data files from Gkeyll simulation"
│   └── nsample: <number of datasets>
│
├── Group: "000001"
│   ├── datasets:
│   │   ├── Fe_mpe_args.txt    # Electron velocity/mu grid parameters
│   │   ├── Fe_mpe.txt         # Electron distribution function at magnetic presheath entrance
│   │   ├── Fi_mpe_args.txt    # Ion velocity/mu grid parameters 
│   │   ├── Fi_mpe.txt         # Ion distribution function at magnetic presheath entrance
│   │   │── input_physparams.txt # Physical parameters for GYRAZE
│   │   └── input_numparams.txt  # Numerical parameters for GYRAZE
│   │
│   └── attributes:
│       ├── x0, y0, z0: spatial coordinates
│       ├── t0: time
│       ├── tf: time frame index
│       ├── alphadeg: field line angle
│       ├── B0, phi0: magnetic field and potential
│       ├── ne0, ni0: electron and ion densities
│       ├── Te0, Ti0: electron and ion temperatures
│       ├── gamma0: rhoe_lambdaD parameter
│       ├── nioverne, TioverTe: density and temperature ratios
│       ├── mioverme, mi, me, e: mass and charge constants
│       └── simprefix: simulation path
│
├── Group: "000002"
│   └── ... (same structure)
│
└── ... (additional groups for each sampled point)
```

#### Usage
You will need to first load your Gkeyll simulation with the `pygkyl` library and then create an instance of `GyrazeInterface`. Below is an example workflow

In [ ]:
# First reset the pygkyl library
import sys
!rm -rf ~/personal_gkyl_scripts/pygkyl/pygkyl.egg-info
!rm -rf ~/personal_gkyl_scripts/pygkyl/build
!{sys.executable} -m pip install ~/personal_gkyl_scripts/pygkyl > ~/personal_gkyl_scripts/pygkyl/install.log

import numpy as np
import matplotlib.pyplot as plt
import os
import pygkyl

simdir = '/Users/ahoffman/personal_gkyl_scripts/sim_data/3x2v/gk_tcv_adapt_src/'
fileprefix = 'rt_gk_tcv_nt_iwl_3x2v_p1'
simulation = pygkyl.simulation_configs.import_config('tcv_nt', simdir, fileprefix)
simulation.normalization.set('fluid velocities','thermal velocity') # fluid velocity moments are normalized by the thermal velocity
simulation.normalization.set('temperatures','eV') # temperatures in electron Volt
simulation.normalization.set('pressures','Pa') # pressures in Pascal
simulation.normalization.set('energies','MJ') # energies in mega Joules
simulation.normalization.set('gradients','major radius') # gradients are normalized by the major radius

sim_frames = simulation.available_frames['ion'] # you can check the available frames for each data type like ion_M0, ion_BiMaxwellian, etc.)
print("%g time frames available (%g to %g)"%(len(sim_frames),sim_frames[0],sim_frames[-1]))

In [ ]:
outfilename = 'gyraze_input_data_test.h5'
gyraze = pygkyl.GyrazeInterface(simulation, number_datasets=True, outfilename=outfilename)
gyraze.generate(
    time_frames = sim_frames[:], 
    xmin = 0.045,
    xmax = 0.115,
    Nxsample = 2,
    Nysample = 2,
    alphadeg = 2.5, 
    zplane = 'both',
    filter_negativity = True,
    verbose = False,
    no_distf = True,
)

In [ ]:
pygkyl.plot_utils.plot_2D_cut(simulation=simulation, 
                              fieldnames=['Bmag','phi','ne','ni','Te','Ti'],
                              cut_dir='xy',
                              cut_coord=0,
                              time_frame=sim_frames[-1],
                              clim=[[1.8,2.3],[0,300],[1e17,3e18],[1e17,5e18],[0,100],[0,300]],
                              xlim =[0.04,0.12]
)

In [ ]:
pygkyl.plot_utils.plot_2D_cut(simulation=simulation, 
                              fieldnames=['fi','fe'],
                              cut_dir='vparmu',
                              time_frame=sim_frames[-1],
                              cut_coord=[0.10,0.0,0])

In [ ]:
pygkyl.plot_utils.plot_1D(simulation=simulation, 
                              fieldnames=['fi','fe'],
                              cdirection='vpar',
                              time_frames=sim_frames[-1],
                              ccoords=[0.10,0.0,0,0.0])

In [ ]:
pygkyl.plot_utils.plot_DG_representation(simulation,
                                         'fi',
                                         sim_frames[-1],
                                         cutdir='vpar',
                                         cutcoord=[0.10,0.0,0,0])

## TODO
- Need to evaluate things at the face not at quadrature points and double check that ther the distribution of ions is 0 for vpar > 0 at z=-Lz/2
- Need to normalize the distribution functions Eq. 133 Gyraze paper

In [ ]:
outfilename = 'gyraze_input_data_test.h5'
gyraze = pygkyl.GyrazeInterface(simulation, number_datasets=True, outfilename=outfilename)
gyraze.generate(
    time_frames = sim_frames[:], 
    xmin = 0.045,
    xmax = 0.115,
    Nxsample = 2,
    Nysample = 2,
    alphadeg = 2.5, 
    zplane = 'both',
    filter_negativity = True,
    verbose = False,
    no_distf=True,
    # lim_dict = {
    #     'phi_norm': {'min': 3.0},
    # }
)

In [ ]:
# We can also load previously generated data
# filename = 'gyraze_input_data.h5'
# gyraze.load_h5_data(filename)
# _ = gyraze.verify_h5_data(verbose=False,Nsamp=24)

In [ ]:
# Export the data contained in the h5 file at a given index as individual files for use in Gyraze
sidx = 0
gyraze.extract_dataset_as_files(group_name=f'{sidx:06d}',output_dir='gyraze_data')

In [ ]:
# Visualize the physical parameters that will be transferred to Gyraze
gyraze.plot_data(alpha=1.0/np.sqrt(400))

In [ ]:
fxy = lambda x: -0.05*x + 7
gyraze.plot_attribute_scatter(attr_x='x0', attr_y='nioverne', color_by='x0', fxy=None)